# Explore microfluidic flow rate and pressure according to a defined geometry

*Date : 20/11/2016*

*Note : Feel free to report any errors/mistakes.*

The purpose of this notebook **is not** to go deep inside the fluid mechanic in microfluidic devices but more trying to develop an intuition of the different physical constants and their order of magnitude at this scale.

![](drawing.png)

For easier calculations, we assume very simple geometries : rectangle or circle.

## Theory

### Flow rate

The flow rate depends on the pressure difference, the properties of the fluid and the geometry used.

#### Cylindrical channel

For a Poiseuille flow, we define the flow rate ($Q$) into a cylindrical channel with :

$
Q = \cfrac{\pi R⁴}{8 \eta} \cfrac{|\Delta P|}{L}
$

Where : 

- $Q$ is flow rate (m³/min)
- $R$ is the internal radius of the cylinder
- $\eta$ is the dynamic fluid viscosity (Pa.s or N.s/m²)
- $L$ is the length of the channel (m)
- $\Delta P$ is difference of pressure (Pa)

#### Rectangular channel

For a Poiseuille flow, we define the flow rate ($Q$) into a rectangular channel with :

$
Q \approx \Bigl[1 - 0.630 \cfrac{h}{w}\Bigr] \cfrac{h³w}{12 \eta L} \Delta P
$

Where : 

- $h$ is the height of the rectangular channel (m)
- $w$ is the width of the rectangular channel (m)

### Flow velocity

The velocity $V$ is calculated from the flow rate ($Q$) and the section ($S$): 

$V = \cfrac{Q}{S}$

Where : 

- $V$ is the fluid velocity (m/s)
- $Q$ is flow rate (m³/s)
- $S$ is surface section (m²)

### Reynolds number

We define the Reynolds number : 

$Re = \cfrac{inertial\; forces}{viscous\; forces} = \cfrac{ρVL}{μ}$

Where : 

- $ρ$ is the density of the fluid (kg/m³)
- $V$ is the velocity of the object relative to the fluid (m/s)
- $L$ is a characteristic length (m)
- $μ$ is the dynamic viscosity of the fluid (Pa.s or N.s/m²)

We can define the kinematic viscosity of the fluid (m²/s) : $ν = \cfrac{μ}{ρ}$. So : 

$Re = \cfrac{ρVL}{μ} = \cfrac{VL}{ν}$

## Calculations

Below some functions to help make the calculation of the different equations above.

In [1]:
import numpy as np

def calculcate_section_circle(diameter):
    return np.pi * ((diameter / 2) ** 2)

def calculcate_section_rectangle(height, width):
    return height * width

def calculate_characteristic_length_circle(diameter):
    return diameter

def calculate_characteristic_length_rectangle(height, width):
    return (2 * height * width) / (height + width)

def calculate_kinematic_viscosity(viscosity, density):
    return viscosity / density

def calculate_flow_rate_circle(diameter, viscosity, length, delta_pressure):
    return (np.pi * (diameter / 2)**4 * np.abs(delta_pressure)) / (8 * viscosity * length)

def calculate_flow_rate_rectangle(height, width, viscosity, length, delta_pressure):
    return (1 - 0.630 * (height / width)) * (((height ** 3) * width * delta_pressure) / (12 * viscosity * length))

def calculate_flow_velocity(flow_rate, section):
    return (flow_rate / section)

def calculate_reynolds_number(kinematic_viscosity, characteristic_length, velocity):
    return (velocity * characteristic_length) / kinematic_viscosity

Define some initial conditions (feel free to test others conditions) :

In [27]:
# Initial conditions

## Geometry
height = 100e-6  # m (not used for circle)
width = 100e-6  # m (not used for circle)
length = 20e-3  # m
diameter = 100e-6  # m (not used for rectangle)
shape = "rectangle"  # Choose in ["rectangle", "circle"]

## Water at 20°C
viscosity = 1.307e-3  # Pa.s or  N.s/m²
density = 1e3  # kg/m³

## Pressure
delta_pressure = 16  # mbar

# Convert to Pa
delta_pressure_pascal = delta_pressure * 100

assert height <= width

Do the math !

In [28]:
if shape == "circle":
    section = calculcate_section_circle(diameter)
    characteristic_length = calculate_characteristic_length_circle(diameter)
    flow_rate = calculate_flow_rate_circle(diameter, viscosity, length, delta_pressure_pascal)
    
elif shape == "rectangle":
    section = calculcate_section_rectangle(height, width)
    characteristic_length = calculate_characteristic_length_rectangle(height, width)
    flow_rate = calculate_flow_rate_rectangle(height, width, viscosity, length, delta_pressure_pascal)
    
kinematic_viscosity = calculate_kinematic_viscosity(viscosity, density)
flow_velocity = calculate_flow_velocity(flow_rate, section)

reynolds_number = calculate_reynolds_number(kinematic_viscosity, characteristic_length, flow_velocity)
    
print("Shape used is '{}'.".format(shape))
print("Difference of pressure is {} mbar.".format(delta_pressure))
print()
print("Section is {0:.3f} mm².".format(section*1e6))
print("Characteristic length is {0:.0f} μm.".format(characteristic_length*1e6))
print()
print("Kinematic viscosity is {} m²/s.".format(kinematic_viscosity))
print()
print("Flow rate is {0:.3f} μl/min.".format(flow_rate*1e9*60))
print("Flow velocity is {0:.3f} m/s.".format(flow_velocity))
print()
print("Reynolds number is {0:.6f}.".format(reynolds_number))

if reynolds_number < 2300:
    print("Flow regime is laminar.")
elif reynolds_number >= 2300 and reynolds_number < 4000:
    print("Flow regime is transitional.")
elif reynolds_number >= 4000:
    print("Flow regime is turbulent.")

Shape used is 'rectangle'.
Difference of pressure is 16 mbar.

Section is 0.010 mm².
Characteristic length is 100 μm.

Kinematic viscosity is 1.307e-06 m²/s.

Flow rate is 11.324 μl/min.
Flow velocity is 0.019 m/s.

Reynolds number is 1.443974.
Flow regime is laminar.


**Important : Calculation is only valid for laminar flow !**

## Source

- https://en.wikipedia.org/wiki/Reynolds_number
- https://en.wikipedia.org/wiki/Pump
- https://en.wikipedia.org/wiki/Pressure-driven_flow
- http://www.dolomite-microfluidics.com/us/downloadsmain/microfluidic-calculator
- http://www.elveflow.com/microfluidic-tutorials/microfluidic-reviews-and-tutorials/flow-control-in-microfluidic-device/
- https://www.uio.no/studier/emner/matnat/math/MEK4450/h11/undervisningsmateriale/modul-5/Pipeflow_intro.pdf
- http://www.pipeflow.co.uk/public/articles/Non_Circular_Pipe_Friction.pdf
- http://pubs.rsc.org/en/content/chapterhtml/2014/bk9781849736718-00001?isbn=978-1-84973-671-8